### 取得股票基本資料

In [1]:
import talib
import numpy as np
import pandas as pd

folder = '/Volumes/Macintosh HD/history/items/price'

columns = ['Open', 'High', 'Low', 'Close', 'Volume']
files = ['開盤價', '最高價', '最低價' ,'收盤價', '成交筆數']

stock_id = '2412'

df = pd.DataFrame()

for idx, filename in enumerate(files):
    df_temp = pd.read_pickle(f"{folder}/{filename}.pkl")
    s = df_temp[stock_id]
    df[columns[idx]] = s
    df[columns[idx]].fillna(method='ffill', inplace=True)

df['Name'] = stock_id

### ADLine - 進貨/出貨線 (Accumulation/Distribution Line)
```ADLine = ((收盤價-最低價)-(最高價-收盤價)) / (最高價 - 最低價) * 成交量```

In [2]:
df_false = df['High'] - df['Low'] != 0
df.loc[df_false, 'ad'] = ((df['Close'] - df['Low']) - (df['High'] - df['Close'])) / (df['High'] - df['Low'] ) * df['Volume']

df_true = df['High'] - df['Low'] == 0
df.loc[df_true,'ad']= df['Close'] / df.shift(1)['Close'] - 1

In [3]:
AD = talib.AD(df['High'],df['Low'], df['Close'], df['Volume'])

df['talib-ad'] = AD

### ADOSC

In [4]:
adosc = talib.ADOSC(df['High'],df['Low'], df['Close'], df['Volume'], fastperiod=3, slowperiod=10)

df['talib-adosc'] = adosc

### ADX - 平均趨向指數（Average Directional Movement Index)


ADX 須配合 +DI & -DI 一起判斷

In [5]:
adx_period = 14

ADX = talib.ADX(df['High'], df['Low'], df['Close'], adx_period)
df['talib-adx'] = ADX

In [6]:
# TR

df_adx = df[['High','Low','Close']].copy()

tr_1 = abs(df_adx['High'] - df_adx['Close'].shift(1))
tr_2 = abs(df_adx['Close'].shift(1) - df_adx['Low'])
tr_3 = abs(df_adx['High'] - df_adx['Low'])

tr_period = f'tr{adx_period}'

temp = pd.DataFrame( { 'tr1': tr_1, 'tr2': tr_2 , 'tr3': tr_3 }) 
df_adx['tr'] = temp.max(axis=1)
df_adx[tr_period] = df_adx['tr'].rolling(adx_period).mean().shift(1-adx_period)
df_adx[tr_period] = df_adx[tr_period].fillna(value=0)

for i, (index, row) in enumerate(df_adx.iterrows()):
    if i == 0:
        continue
    row[tr_period] = (df_adx.iloc[i-1][tr_period] * (adx_period - 1) / adx_period) + row.tr * 1 / adx_period

In [7]:
# DM
df_adx['dm_positive'] = df_adx['High'] - df_adx['High'].shift(1)
df_adx['dm_negative'] = df_adx['Low'].shift(1) - df_adx['Low']

def dm_apply(row):
    # 若+DM>-DM成立，且+DM大於0
    if row['dm_positive'] < 0 or row['dm_positive'] <= row['dm_negative']:
        row['dm_positive'] = 0
    # 若+DM<-DM成立，且-DM大於0
    if row['dm_negative'] < 0 or row['dm_negative'] <= row['dm_positive']:
        row['dm_negative'] = 0
        
    return row
    
df_adx = df_adx.apply(dm_apply, axis=1)

# DM(14)
dmp_period = f'dm_positive{adx_period}'
dmn_period = f'dm_negative{adx_period}'

df_adx[dmp_period] = df_adx['dm_positive'].rolling(adx_period).mean().shift(1-adx_period)
df_adx[dmn_period] = df_adx['dm_negative'].rolling(adx_period).mean().shift(1-adx_period)
df_adx[[dmp_period, dmn_period]] = df_adx[[dmp_period,dmn_period]].fillna(value=0)

for i, (index, row) in enumerate(df_adx.iterrows()):
    if i == 0:
        continue
    
    row[dmp_period] = \
        (df_adx.iloc[i-1][dmp_period] * (adx_period - 1) / adx_period) + row.dm_positive * 1 / adx_period
    row[dmn_period] = \
        (df_adx.iloc[i-1][dmn_period] * (adx_period - 1) / adx_period) + row.dm_negative * 1 / adx_period

In [8]:
# DI
dip_period = f'di_positive{adx_period}'
din_period = f'di_negative{adx_period}'
df_adx[dip_period] = df_adx[dmp_period] / df_adx[tr_period] * 100
df_adx[din_period] = df_adx[dmn_period] / df_adx[tr_period] * 100

# DX
df_adx['dx'] = abs(df_adx[dip_period] - df_adx[din_period]) \
                / (df_adx[dip_period] + df_adx[din_period]) \
                * 100

df_adx['dx'] = df_adx['dx'].fillna(0)

# ADX
df_adx['adx'] = df_adx['dx'].rolling(adx_period).mean().shift(1-adx_period)
df_adx['adx'] = df_adx['adx'].fillna(value=0)

for i, (index, row) in enumerate(df_adx.iterrows()):
    if i == 0:
        continue
    
    row.adx = (df_adx.iloc[i-1]['adx'] * (adx_period - 1) / adx_period) + row.dx * 1 / adx_period

df['adx'] = df_adx['adx']

### ADXR

In [11]:
df['talib-adxr'] = talib.ADXR(df['High'], df['Low'], df['Close'], timeperiod=14)